In [ ]:
import datetime
print(datetime.datetime.now())
today = datetime.date.today()
yesterday = today - datetime.timedelta(1)
target = "{}-{}".format(yesterday.month, yesterday.day)

f = open("./per_day_keyword/(keyword)" + target + ".txt", mode = "r", encoding="utf-8")
text = f.read()
exec("text = " + text)
f.close()

from collections import Counter
count = Counter(text)
keywords = count.most_common(50)

# 키워드, 개수, 하루 총 단어 수 저장
keys = [ item[0] for item in keywords ]
counts = [ item[1] for item in keywords ]

# HTML 생성 시 쓰이는 것.
f = open("./per_day_after/(after)" + target + ".txt", mode = "r", encoding="utf-8-sig")
data = f.read()
exec("data = " + data)
f.close()

counts.append(len(data))
keycount = keys + counts


# 머신러닝 토픽 추출
import random

def p_topic_given_document(topic, d, alpha=0.1):
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

documents = [[]]

for i in range(len(data)):
    data[i] = data[i][0]
    documents.append([])

for i in range(len(data)):
    for words in data[i]:
        if len(words) >= 2:
            documents[i].append(words)

random.seed(486)
K=50
document_topics = [[random.randrange(K) for word in document] for document in documents]
document_topic_counts = [Counter() for _ in documents]
topic_word_counts = [Counter() for _ in range(K)]
topic_counts = [0 for _ in range(K)]
document_lengths = [len(document) for document in documents]
distinct_words = set(word for document in documents for word in document)
V = len(distinct_words)
D = len(documents)

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for iter in range(10):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d], document_topics[d])):
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

temp=[]
temp2=[]
for i in topic_word_counts:
    temp.append(i.most_common(20))

for i in range(len(temp)):
    temp2.append([])
    for j in temp[i]:
        temp2[i].append(list(j))

######################↑토픽추출##########################        
######################↓출력처리##########################
data_original = temp2

data_only_topic=[]
for i in range(len(data_original)):
    data_only_topic.append([])
    for j in range(len(data_original[i])):
        data_only_topic[i].append(data_original[i][j][0])
# print(data_only_topic)

data_dict=[]
for i in range(len(data_original)):
    data_dict.append({})
    for j in range(len(data_original[i])):
        data_dict[i][data_original[i][j][0]] = data_original[i][j][1]
# print(data_dict)

topic_index=[]
for i in range(len(data_only_topic)):
    topic_index.append([])
    for j in range(len(data_only_topic[i])):
        topic_index[i].append({})
        for k in range(len(data_dict)):
            if data_only_topic[i][j] in data_dict[k]:
                topic_index[i][j][k] = data_dict[k][data_only_topic[i][j]]
# print(topic_index)

words_topic={}

for i in range(len(data_only_topic)):
    for j in range(len(data_only_topic[i])):
        words_topic[data_only_topic[i][j]] = data_only_topic[max(topic_index[i][j], key=lambda k: topic_index[i][j][k])]
# print(words_topic)

result = {}
for key in words_topic :
    sentence = ""
    for item in words_topic[key][0:5] :
        sentence += item + ', '
    result[key] = sentence[:-2]

    
# keycount 표를 만들어서 html 저장
html_source = """
<!DOCTYPE HTML>
<html>
<head>
  <meta charset="utf-8">
  <style>
    a { cursor:pointer; }
    table, th { text-align:center !important; }
  </style>
  <script
    src="https://code.jquery.com/jquery-3.3.1.min.js"
    integrity="sha256-FgpCb/KJQlLNfOu91ta32o/NMZxltwRo8QtmkMRdAu8="
    crossorigin="anonymous"></script>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.2/css/bootstrap.min.css">
    <script>
    $(document).ready(function (){
      var text = """ + str(keycount) + ';' + """
      var topic = """ + str(result) + ';' + """
      for(var i = 0; i < 50; i++)
      {
        $('#rank').append(
          $('<tr>').append(
            "<td>" + (i+1) + "</td>" +
            "<td><a class='keyword'>" + text[i]  + "</a></td>" +
            "<td>" + (text[50+i] / text[100] * 100).toFixed(3) +  "%</td>" +
            "<td>" + topic[text[i]] + "</td>"
          )
        );
      }
      var keyword =document.getElementsByClassName('keyword');
      for (let i = 0; i < keyword.length; i++) {
        keyword[i].onclick = function() {
          window.parent.postMessage(keyword[i].innerHTML, '*');
        }
      }
    });
  </script>
</head>
<body>
  <table id="rank" class="table-condensed table-bordered table-striped" align = "center">
    <th style="width:100px">순위</th>
    <th style="width:250px">키워드</th>
    <th style="width:150px">언급 비율</th>
    <th style="width:400px">관련 키워드</th>
  </table>
</body>
</html>
"""
f = open("./rank/table-" + target + ".html", mode = 'w', encoding = "utf-8")
f.write(str(html_source))
f.close()

print(datetime.datetime.now())


# 버킷에 업로드
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))
    
bucket_name = 'gunmin_test'
source_file_name = "./rank/table-" + target + ".html"
destination_blob_name = "rank/table-" + target + ".html"

upload_blob(bucket_name, source_file_name, destination_blob_name)